In [19]:
# 107303538_1118_hw10_1

In [20]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim # 最佳化資料庫

數據轉tensor，已知t_c 未知t_u

In [21]:
# 數據
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0] # 攝氏溫度參數
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # 未知參數(待得知關係的)
t_c = torch.tensor(t_c).unsqueeze(1) # 變成兩軸處理 直行 [ ,1]
t_u = torch.tensor(t_u).unsqueeze(1)
t_u.shape

torch.Size([11, 1])

小模型linear_model　(內建parameters與lr)

In [22]:
# 定義小model linear_model
linear_model = nn.Linear(1, 1) # 一個input和output (input, output)
# 想成nn.Linear為已經訓練好的model 內建parameters與lr

小模型最佳化器( parameters() + Ir )

In [23]:
# 小模型優化器
import torch.optim as optim # 最佳化資料庫
optimizer = optim.SGD( # SDG方法
  linear_model.parameters(), # 在linear_model裡面的parameters中已經有w和b
  lr=1e-2) # learning rate為1e-2

In [24]:
# 可略
# list(linear_model.parameters()) # 看parameter內w b的值 用list把(w,b)包起來 每次都會不一樣 (已預設requires_grad=True) 

主模型seq_model(未命名)

In [25]:
"""
# 定義類神經網路主model seq_model 可以直接跳到取有意義名字的地方
seq_model = nn.Sequential( # Sequential為有順序性的訓練
  nn.Linear(1, 13), # 有13個線路 (輸入1個數據 訓練完輸出13個) 為hidden layer
  nn.Tanh(), # 經過非線性的激活函數處理 神經網路 
  nn.Linear(13, 1)) # 13個線路 (輸出13個圈圈 再轉換變1個)
seq_model
"""

'\n# 定義類神經網路主model seq_model 可以直接跳到取有意義名字的地方\nseq_model = nn.Sequential( # Sequential為有順序性的訓練\n  nn.Linear(1, 13), # 有13個線路 (輸入1個數據 訓練完輸出13個) 為hidden layer\n  nn.Tanh(), # 經過非線性的激活函數處理 神經網路 \n  nn.Linear(13, 1)) # 13個線路 (輸出13個圈圈 再轉換變1個)\nseq_model\n'

In [26]:
# 可略
# [param.shape for param in seq_model.parameters()] # 用for跑 如果想知道大模型seq_model.parameters()中每一個param (w b)的shape

In [27]:
# 原名字 named_parameters()
#for name, param in seq_model.named_parameters(): # seq_model中每個線路都有各自的w b 一區的線路(一組nn.Linear( , )使用一個)數字標號
#  print(name, param.shape)

主模型seq_model(命名)

In [28]:
# 希望可以取有意義的名字
from collections import OrderedDict # 用dict的方式命名
seq_model = nn.Sequential(OrderedDict([ # OrderedDict
  # Sequential為有順序性的訓練
  # [key, value]
  ('hidden_linear', nn.Linear(1, 13)), # 把第一區線路取名為hidden_linear
  ('hidden_activation', nn.Tanh()), # 激活函數為hidden_activation
  ('output_linear', nn.Linear(13, 1)) # 輸出方的線路為output_linear
]))
seq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=13, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=13, out_features=1, bias=True)
)

看新取的名字

In [29]:
# 新的名字
for name, param in seq_model.named_parameters(): 
  print(name, param.shape)

hidden_linear.weight torch.Size([13, 1])
hidden_linear.bias torch.Size([13])
output_linear.weight torch.Size([1, 13])
output_linear.bias torch.Size([1])


主模型最佳化器( parameters() + Ir )

In [30]:
# 大模型的優化器
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

In [31]:
# seq_model.output_linear.bias # 取seq_model中名字為output_linear的b

處理train和validation的indice

分成train和validation的數據數目

In [32]:
# 分成train和validation的數據
n_samples = t_u.shape[0] # 共幾筆資料
n_val = int(0.2 * n_samples) # n_val 設為驗證集 數量為sample的0.2倍
# int( )字符串或數字轉換為整數

隨機分成train和validation的indice

In [33]:
# 處理indice
shuffled_indices = torch.randperm(n_samples) # 把sample變成random 會有更好的學習效果
train_indices = shuffled_indices[:-n_val] # train的indice數 => 從前面取 取到倒數幾個(由驗證集的數量決定) 若驗證集為2則取道倒數-2個
val_indices = shuffled_indices[-n_val:] # val的indice數 => 從倒數後幾個取 取到這個list底
train_indices, val_indices

(tensor([3, 9, 5, 0, 7, 8, 2, 1, 6]), tensor([10,  4]))

處理train和validation的數據

In [34]:
train_t_u = t_u[train_indices] # c和u中要被train的
train_t_c = t_c[train_indices]
val_t_u = t_u[val_indices] # c和u中要被驗證val的
val_t_c = t_c[val_indices]
train_t_un = 0.1 * train_t_u # normalize讓效果變好(針對u待測物)
val_t_un = 0.1 * val_t_u

training_loop ( 多model, loss_fn 少params )

In [35]:
# 定義training_loop (不太會變)
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val): 
  for epoch in range(1, n_epochs + 1):

    # train
    t_p_train = model(t_u_train) # u就進去model跑
    loss_train = loss_fn(t_p_train, t_c_train) # c就和u結果比較loss

    # val
    t_p_val = model(t_u_val) # val組同理
    loss_val = loss_fn(t_p_val, t_c_val)

    # 必要三步驟
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0: # 只看1和1000組的倍數
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
          f" Validation loss {loss_val.item():.4f}")

訓練開始(最後回傳w b)

In [36]:
training_loop(
  n_epochs = 5000,
  optimizer = optimizer,
  model = seq_model, # 設定為seq_model
  loss_fn = nn.MSELoss(), # 內建有nn.MSELoss()去計算loss
  t_u_train = train_t_un, # 正規化後的數據u
  t_u_val = val_t_un,
  t_c_train = train_t_c, # 原始數據c
  t_c_val = val_t_c)
print('output', seq_model(val_t_un))
print('answer', val_t_c)
print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 178.0102, Validation loss 300.5553
Epoch 1000, Training loss 4.1890, Validation loss 3.3408
Epoch 2000, Training loss 2.3318, Validation loss 1.6532
Epoch 3000, Training loss 2.0928, Validation loss 1.2668
Epoch 4000, Training loss 2.0453, Validation loss 1.1431
Epoch 5000, Training loss 2.0276, Validation loss 1.0811
output tensor([[20.6097],
        [12.4177]], grad_fn=<AddmmBackward0>)
answer tensor([[21.],
        [11.]])
hidden tensor([[ 0.0476],
        [ 0.0102],
        [-0.0045],
        [-0.0065],
        [-0.0013],
        [-0.0068],
        [-0.0035],
        [-0.0056],
        [-0.0100],
        [-0.0053],
        [ 0.0238],
        [ 0.0041],
        [ 0.0023]])
